In [ ]:
from keras.initializers import glorot_uniform
from keras.layers import Input, Dense, Activation, Add
from keras.models import Model
from keras.optimizers import Adam, SGD

from tensorflow.keras.utils import to_categorical

from sklearn import datasets
from sklearn.model_selection import train_test_split

import seaborn as sns
from matplotlib import pyplot as plt

import numpy as np
from mlxtend.plotting import plot_decision_regions

sns.set(style="whitegrid")
%config InlineBackend.figure_format='retina'

In [ ]:
NUM_CLASSES = 8

X,y=datasets.make_blobs(n_samples=2000, 
                        shuffle=True, random_state=2, 
                        centers=NUM_CLASSES,
                        cluster_std=0.75,
                        )
y_cat = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.3, random_state=0)

plt.scatter(X[:,0],X[:,1],c=y)
plt.show()

In [ ]:
def identity_block(X, stage):

  name_base = 'stage_' + str(stage)  + '_'

  X_shortcut = X

  X = Dense(8, name=name_base + 'dense_a')(X)
  X = Activation('relu', name=name_base + 'act1')(X)
  # X = Dense(8, name=name_base + 'dense_b')(X)
  X = Add(name=name_base + 'add')([X, X_shortcut])
  X = Activation('relu', name=name_base + 'act2')(X)

  return X

def FakeResNet(input_shape=(2)):

  X_input = Input(input_shape)
  X = Dense(8, name='stage_0_dense', activation='relu')(X_input)

  X = identity_block(X, 1)
  X = identity_block(X, 2)  
  X = identity_block(X, 3)
  
  model = Model(inputs=X_input, outputs=X, name='FakeResNet')

  return model

In [ ]:
base_model = FakeResNet(input_shape=(2,))

headModel = base_model.output
headModel = Dense( NUM_CLASSES,activation='softmax', name='fc3',kernel_initializer=glorot_uniform(seed=0))(headModel)

model = Model(inputs=base_model.input, outputs=headModel)

model.compile(
    optimizer=SGD(learning_rate=0.01, momentum=0.9), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'],
    )

model.summary()

In [ ]:
H = model.fit(X_train, y_train, 
              validation_split=0.1,
              epochs=200, verbose=1, 
              )

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
class Onehot2Int(object):

    def __init__(self, model):
        self.model = model

    def predict(self, X):
        y_pred = self.model.predict(X)
        return np.argmax(y_pred, axis=1)

# fit keras_model
model_cat = Onehot2Int(model)

# Plot decision boundary
plot_decision_regions(X_test, np.argmax(y_test, axis=1), clf=model_cat, legend=0)
plt.show()